In [ ]:
!pip install requests


In [ ]:
!curl -H "Authorization: token *personal-access-token-from-github*" \
"https://api.github.com/search/users?q=location:Hyderabad+followers:>50?per_page=100" \
-o users.json

!curl -L \
  -H "Accept: application/vnd.github+json" \
  -H "Authorization: Bearer *personal-access-token-from-github*" \
  "https://api.github.com/search/users?q=location:Hyderabad+followers:>50"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   109  100   109    0     0    654      0 --:--:-- --:--:-- --:--:--   656
{
  "message": "Bad credentials",
  "documentation_url": "https://docs.github.com/rest",
  "status": "401"
}


In [ ]:
import requests
import pandas as pd
import time

GITHUB_TOKEN =  '*personal-access-token-from-github*'
HEADERS = {'Authorization': f"token {GITHUB_TOKEN}",
           "Accept": "application/vnd.github.v3+json"
}
BASE_URL = "https://api.github.com/rate_limit"
response = requests.get(BASE_URL, headers=HEADERS)
print(response.json())


**TDS Project 1 - Process the files before answering the assignment questions**

In [ ]:
import pandas as pd
import requests
import time

# Example endpoint to get user data from Hyderabad with more than 50 followers
GITHUB_TOKEN =  '*personal-access-token-from-github*'
HEADERS = {'Authorization': f"token {GITHUB_TOKEN}",
           "Accept": "application/vnd.github.v3+json"
}
BASE_URL = "https://api.github.com"
location = "Hyderabad"
min_followers = 50
cur_page = 1
all_users = []

while True:
  search_url = f"{BASE_URL}/search/users?q=location:{location}+followers:>{min_followers}&per_page=100&page={cur_page}"
  response = requests.get(search_url, headers=HEADERS)
  print(response.json())

  if response.status_code!=200:
    print(f"Error fetching page {cur_page}: {response.status_code}")

  else:
    data = response.json()
    users = data['items']
    print(users)

    if not users:
      break
    all_users.extend(users)
    if len(users) < 100:
      break

    time.sleep(1)
    cur_page += 1

if len(all_users) > 0:
  print(f"Total users found: {len(all_users)}")
  df = pd.DataFrame(all_users)
  df.to_csv('users.csv', index=False)

Total users found: 505


In [ ]:
import pandas as pd
import requests
import time

# Example endpoint to get user data from Hyderabad with more than 50 followers
GITHUB_TOKEN =  '*personal-access-token-from-github*'
HEADERS = {'Authorization': f"token {GITHUB_TOKEN}",
           "Accept": "application/vnd.github.v3+json"
}

BASE_URL = "https://api.github.com"

df = pd.read_csv('users.csv')

for user in df['login']:
  cur_user = user

search_url = f"{BASE_URL}/users/{cur_user}/repos?per_page=1&page=1"
response = requests.get(search_url, headers=HEADERS)
print(response.json())

In [ ]:
import pandas as pd
import requests
import time

# Example endpoint to get user data from Hyderabad with more than 50 followers
GITHUB_TOKEN =  '*personal-access-token-from-github*'
HEADERS = {'Authorization': f"token {GITHUB_TOKEN}",
           "Accept": "application/vnd.github.v3+json"
}
BASE_URL = "https://api.github.com"
count_per_page = 100
current_page = 1
all_repos = []

df = pd.read_csv('users.csv')

#for user in range(min(5, len(df['login']))):
  #print("User: ",df['login'].iloc[user])

for user in df['login']:
  print(user)
  cur_user = user

  while True:
   search_url = f"{BASE_URL}/users/{cur_user}/repos?per_page={count_per_page}&page={current_page}"
   response = requests.get(search_url, headers=HEADERS)

   if response.status_code != 200:
    print(f"Error fetching page {current_page}: {response.status_code}")

   repos = response.json()
   if(len(repos) > 0):
    all_repos.extend(repos)

   if(len(repos) < count_per_page):
    break
   else:
    time.sleep(0.1)
    current_page += 1

print(f"Total repos found: {len(all_repos)}")
if all_repos:
  df = pd.DataFrame(all_repos)
  df.to_csv('repositories.csv', index=False)
else:
  print("No repositories found to create a DataFrame.")

In [ ]:
import pandas as pd
import requests
import time

# Example endpoint to get user data from Hyderabad with more than 50 followers
GITHUB_TOKEN =  '*personal-access-token-from-github*'
HEADERS = {'Authorization': f"token {GITHUB_TOKEN}",
           "Accept": "application/vnd.github.v3+json"
}

BASE_URL = "https://api.github.com"
df = pd.read_csv('users.csv')
final_users = []

for user in df['login']:
  cur_user = user

  search_url = f"{BASE_URL}/users/{cur_user}"
  response = requests.get(search_url, headers=HEADERS)
  user_data = response.json()

  #Extract the fields you need - login,name,company,location,email,hireable,bio,public_repos,followers,following,created_at:
  # cleaned_data = {
  #     'login': user_data['login'],
  #     'name': user_data['name'],
  #     'company': user_data['company']
  # }

  final_users.append(user_data)

print(final_users)

df = pd.DataFrame(final_users)
df.to_csv('user_info.csv', index=False)

**USERS:**

In [ ]:
import requests
import pandas as pd
from io import StringIO

# URL to access the raw CSV file directly
url_users = "https://raw.githubusercontent.com/Rajalakshmi12/IITM_Tds_Project1/main/users.csv"
url_repos = "https://raw.githubusercontent.com/Rajalakshmi12/IITM_Tds_Project1/main/repositories.csv"

#Q1,Q2,Q4 Make the request to get the CSV file content
response = requests.get(url_users)

if response.status_code == 200:
    # Read the CSV content into a pandas DataFrame
    csv_content = response.content.decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    df['created_at'] = pd.to_datetime(df['created_at'], utc=True)

    #Q1. Top 5 users
    df.index = df.index + 1
    df_sorted1 = df.sort_values(by="followers", ascending=False)
    #print(df_sorted1['login'].head(5))

    #Q2. Top 5 early registered Users
    df_sorted2 = df.sort_values(by="created_at", ascending=True)
    #print(df_sorted2[['login']].head(5))

    #Q4. Top company these developers work at, group by company and sort by count in desc order
    grouped_df4 = df.groupby('company').size().reset_index(name='count')
    df_sorted4 = grouped_df4.sort_values(by="count", ascending=False)
    #print(df_sorted4)

    #Q6. Get the users joined after 2020, then get the second most popular language
    df['login'] = df['login'].str.strip()
    target_datetime = pd.Timestamp('2019-12-31 23:59:59+00:00',tz='UTC')
    df_filt6 = df[df['created_at'] > target_datetime]
    usernames = df_filt6['login'].tolist()
    #Get the list of usernames from the filtered list
    print("Length of usernames: ",len(usernames))

    #80 users repositories languages required, so get the login id and filter the data
    response_from_repos = requests.get(url_repos)

    if response_from_repos.status_code == 200:
      # Read the CSV content into a pandas DataFrame
      csv_content = response_from_repos.content.decode('utf-8')
      df_repo = pd.read_csv(StringIO(csv_content))
      df_repo['login'] = df_repo['login'].str.strip()

      for user in usernames:
        df_repofilt_temp = df_repo[df_repo['login'] == user]
        if not df_repofilt_temp.empty:
          print(f"User '{user}' found:")

else:
    print(f"Error: {response.status_code}, {response.text}")

**REPOSITORIES:**

In [ ]:
import requests
import pandas as pd
from io import StringIO

#Q3 Make the request to get the CSV file content
url_repos = "https://raw.githubusercontent.com/Rajalakshmi12/IITM_Tds_Project1/main/repositories.csv"
response = requests.get(url_repos)

if response.status_code == 200:
    # Read the CSV content into a pandas DataFrame
    csv_content = response.content.decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))
    df['created_at'] = pd.to_datetime(df['created_at'])
    df = df.dropna(subset=['license_name'])
    df.index = df.index + 1


    #Q3 Top 3 most popular license --> Sort count by Descending order
    #Group by code
    grouped_df3 = df.groupby('license_name').size().reset_index(name='count')
    df_sorted3 = grouped_df3.sort_values(by='count', ascending=False)
    #print(df_sorted3)

    #Q5 Most popular programming language
    #Group by Language
    grouped_df5 = df.groupby('language').size().reset_index(name='count')
    df_sorted5 = grouped_df5.sort_values(by='count', ascending=False)
    #print(df_sorted5)

    df = df.dropna(subset=['language'])

    #Q7.Which language has the highest average number of stars per repository?
    #Group by Programming Language
    #Collect all the starts count and find the average
    grouped_df7 = df.groupby('language')['stargazers_count'].mean().reset_index(name='average_count')

    #Sort by average count
    df_sorted7 = grouped_df7.sort_values(by='average_count', ascending=False)
    print(df_sorted7)


In [ ]:
import requests
import pandas as pd
from io import StringIO

#Q3 Make the request to get the CSV file content
url_users = "https://raw.githubusercontent.com/Rajalakshmi12/IITM_Tds_Project1/main/users.csv"
url_repos = "https://raw.githubusercontent.com/Rajalakshmi12/IITM_Tds_Project1/main/repositories.csv"
response = requests.get(url_repos)
response_users = requests.get(url_users)

if response.status_code == 200:
    # Read the CSV (Repos) content into a pandas DataFrame
    csv_content = response.content.decode('utf-8')
    df = pd.read_csv(StringIO(csv_content))

    #Q6. Which programming language is the second most popular among users who joined after 2020?

    #Group by code
    df_fltr = df[df['login'] == 'maddydevgits']
    grouped_dft6 = df_fltr.groupby('language').size().reset_index(name='count')
    df_sortedt6 = grouped_dft6.sort_values(by='count', ascending=False)

    #Q14. Who created the most repositories on weekends (UTC)? List the top 5 users' login in order, comma-separated

    df['created_at'] = pd.to_datetime(df['created_at'])
    df['day_of_week'] = df['created_at'].dt.dayofweek

    df_fltr_14 = df[df['day_of_week'].isin([5,6])]
    grouped_dft14 = df_fltr_14.groupby('login').size().reset_index(name='count')
    df_sortedt14 = grouped_dft14.sort_values(by='count', ascending=False)
    print(df_sortedt14)


    #Q9. What is the correlation between the number of followers and the number of public repositories among users in Hyderabad?

if response_users.status_code == 200:
    # Read the CSV (Repos) content into a pandas DataFrame
    csv_content = response_users.content.decode('utf-8')
    df_users = pd.read_csv(StringIO(csv_content))
    #print(df_users[['followers','count_of_repos']])

    #Correlation between Followers and repos count:
    #for col in df_users.columns:
     # if 'followers' in col or 'count' in col:
    correlation_coefficient = df_users['followers'].corr(df_users['count_of_repos'])

    # Print the correlation coefficient
    print(f"Correlation coefficient between followers and total repositories: {correlation_coefficient}")

            login  count
2      anjijava16     78
1    Shekharrajak     76
3       arvindr21     68
6     in28minutes     25
7    maddydevgits     25
0      MadhavBahl     13
5  iam-veeramalla      6
4   ashokitschool      5
Correlation coefficient between followers and total repositories: 0.22979333250595757


In [ ]:
import pandas as pd

#Q9. What is the correlation between the number of followers and the number of public repositories among users in Hyderabad?
users_df = pd.read_csv('users.csv')

# Step 4: Calculate the correlation between 'followers' and 'public_repos'
correlation_coefficient = users_df['followers'].corr(users_df['public_repos'])

# Print the correlation coefficient
print(f"Correlation coefficient between followers and public repositories among users in Hyderabad: {correlation_coefficient}")


Correlation coefficient between followers and public repositories among users in Hyderabad: 0.005667047286286638


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

#Q10. Does creating more repos help users get more followers? Using regression, estimate how many additional followers a user gets per additional public repository.
users_df = pd.read_csv('users.csv')

# Step 4: Calculate the correlation between 'followers' and 'public_repos'
x = users_df[['public_repos']].values
y = users_df['followers'].values

print(x.shape)
print(y.shape)

# Fit a linear regression model
model = LinearRegression()
model.fit(x,y)

print(model.coef_[0])

#Slope = 0.06296

(503, 1)
(503,)
0.06296441574945554


In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np

#Q11. Do people typically enable projects and wikis together? What is the correlation between a repo having projects enabled and having wiki enabled?
users_df_11 = pd.read_csv('repositories.csv')
users_df_11 = users_df_11.dropna(subset=['has_projects', 'has_wiki'])

# Step 4: Calculate the correlation between 'followers' and 'public_repos'
users_df_11['has_projects'] = users_df_11['has_projects'].astype(int)
users_df_11['has_wiki'] = users_df_11['has_wiki'].astype(int)

# Print the correlation coefficient
correlation_11 = users_df_11['has_projects'].corr(users_df_11['has_wiki'])
print(f"Correlation coefficient between has_projects and has_wiki: {correlation_11}")


Correlation coefficient between has_projects and has_wiki: nan


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
from re import S
import pandas as pd
import requests
from sklearn.linear_model import LinearRegression
import numpy as np
from io import StringIO


#Q12. Do hireable users follow more people than those who are not hireable?
# Average of following per user for hireable=true minus the average following for the rest (to 3 decimal places, e.g. 12.345 or -12.345)
url_users = "https://raw.githubusercontent.com/Rajalakshmi12/IITM_Tds_Project1/main/users.csv"
response = requests.get(url_users)

if response.status_code == 200:
    # Read the CSV (Repos) content into a pandas DataFrame
    csv_content = response.content.decode('utf-8')
    users_df = pd.read_csv(StringIO(csv_content))
    users_df['hireable'] = users_df['hireable'].astype(bool)

    users_dffilter12_q1 = users_df[users_df['hireable'] == True]
    first_mean = users_dffilter12_q1['following'].mean()
    print(round(first_mean,3))

    users_dffilter12_q2 = users_df[users_df['hireable'] == False]
    second_mean = users_dffilter12_q2['following'].mean()
    print(round(second_mean,3))

    average_difference = round(round(first_mean,3) - round(second_mean, 3),3)
    print(average_difference)

#Q15. Do people who are hireable share their email addresses more often?
    #users_df = users_df.dropna(subset=['email'])
    users_dffilter15_v1 = users_df[users_df['hireable'] == True]
    users_dffilter15_v2 = users_df[users_df['hireable'] == False]

    frac_hire = users_dffilter15_v1['email'].count()/users_dffilter15_v1.shape[0]
    frac_non_hire = users_dffilter15_v2['email'].count()/users_dffilter15_v2.shape[0]
    print(round(frac_hire,3))
    print(round(frac_non_hire,3))
    print(round(frac_hire - frac_non_hire,3))

    #Q13. Some developers write long bios. Does that help them get more followers? What's the correlation of the length of their bio (in Unicode words, split by whitespace) with followers? (Ignore people without bios)
    users_df = users_df.dropna(subset=['bio'])
    users_df['bio_length'] = users_df['bio'].str.split().str.len()

    x = users_df[['bio_length']].values
    y = users_df['followers'].values
    model = LinearRegression()
    model.fit(x,y)
    print(model.coef_[0])

    #!16. Let's assume that the last word in a user's name is their surname (ignore missing names, trim and split by whitespace.) What's the most common surname? (If there's a tie, list them all, comma-separated, alphabetically)
    users_df = users_df.dropna(subset=['name'])
    users_df['surname'] = users_df['name'].str.split().str[-1]

    grouped_16 = users_df.groupby('surname').size().reset_index(name='count')
    df_sorted16 = grouped_16.sort_values(by='count', ascending=False)
    print(df_sorted16)